# Combining & organizing data

## Libraries and settings

In [1]:
# Libraries
import os
import pandas as pd
import numpy as np
import fnmatch

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Show current working directory
print(os.getcwd())

/Users/john/School/Sem5/DA/data_analytics/Week_03


## Importing the prepared rental apartments data

In [2]:
# Read data to pandas data frame
df_orig = pd.read_csv('apartments_data_prepared.csv', 
                      sep=',', 
                      encoding='utf-8')

# Copy of data with selected colums
columns = [ 'web-scraper-order', 
            'address_raw',
            'rooms', 
            'area', 
            'luxurious',
            'price',
            'price_per_m2']
df = df_orig[columns]

# Get number of rows and columns
print(df.shape)

# Show first records
df.head(5)

(865, 7)


,web-scraper-order,address_raw,rooms,area,luxurious,price,price_per_m2
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",3.0,49.0,0,1441.0,29.41
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",3.5,65.0,0,1850.0,28.46
2,1693998201-4,"Cramerstrasse 8-12, 8004 Zürich, ZH",2.0,54.0,0,4853.0,89.87
3,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",2.0,49.0,0,4335.0,88.47
4,1693998201-16,"Wolframplatz 1, 8045 Zürich, ZH",2.0,32.0,0,3515.0,109.84


## Combining data from different sources

### Reading rental apartment data with geocoded addresses

In [20]:
# Meaning of variables
# lat: geographical latitude
# lon: geographical longitude
# bfs_number: official municipality id
# bfs_name: official municipality name

# Geocoded data (i.e. data with latitude and longitude)
df_geo = pd.read_csv('apartments_data_geocoded.csv', 
                     sep=',', 
                     encoding='utf-8')

# Rename columns
df_geo = df_geo.rename(columns={'web-scrape':'web-scraper-order',
                                'address_ra':'address_raw',
                                'BFS_NUMMER':'bfs_number', 
                                'NAME':'bfs_name'})

# Show data
df_geo[['web-scraper-order',
        'address_raw',
        'lat', 
        'lon', 
        'bfs_number', 
        'bfs_name']].head()

,web-scraper-order,address_raw,lat,lon,bfs_number,bfs_name
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",47.252171,8.845797,118,Rüti (ZH)
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",47.513332,8.474851,88,Neerach
2,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",47.370792,8.514748,261,Zürich
3,1693998201-16,"Wolframplatz 1, 8045 Zürich, ZH",47.362282,8.522193,261,Zürich
4,1693998205-25,"Badenerstrasse 67, 8953 Dietikon, ZH",47.407925,8.392561,243,Dietikon


### Join geo-information to rental apartment data using .merge()

In [21]:
df2 = df.merge(df_geo[['web-scraper-order', 
                       'lat', 
                       'lon', 
                       'bfs_number', 
                       'bfs_name']], 
               on="web-scraper-order")
df2.head()

,web-scraper-order,address_raw,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",3.0,49.0,0,1441.0,29.41,47.252171,8.845797,118,Rüti (ZH)
1,1693998201-2,"Zürcherstrasse 1, 8173 Neerach, ZH",3.5,65.0,0,1850.0,28.46,47.513332,8.474851,88,Neerach
2,1693998201-5,"Rotachstrasse 33, 8003 Zürich, ZH",2.0,49.0,0,4335.0,88.47,47.370792,8.514748,261,Zürich
3,1693998201-16,"Wolframplatz 1, 8045 Zürich, ZH",2.0,32.0,0,3515.0,109.84,47.362282,8.522193,261,Zürich
4,1693998205-25,"Badenerstrasse 67, 8953 Dietikon, ZH",3.5,105.0,0,3200.0,30.48,47.407925,8.392561,243,Dietikon


### Reading municipality-level data

In [22]:
# Meaning of variables:
# bfs_number: official municipality id
# bfs_name: official municipality name
# pop: number of residents (=population)
# pop_dens: population density (pop per km2)
# frg_pct: percentage foreigners
# emp: numer of employees
# mean_taxable_income: mean taxable income per resident

df_municip = pd.read_excel('municipality_data.xlsx', 
                           sheet_name='data_for_import')
df_municip.head(5)

,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,mean_taxable_income
0,1,Aeugst am Albis,1981,250.442478,14.184755,442.0,105013.570634
1,2,Affoltern am Albis,12303,1161.756374,28.700317,6920.0,71059.805603
2,3,Bonstetten,5572,749.932705,16.564968,1014.0,88927.698145
3,4,Hausen am Albis,3751,275.808824,16.022394,1021.0,86300.455137
4,5,Hedingen,3778,578.560490,16.410799,1478.0,90811.205330


### Join municipality data to rental apartment data using .merge()

In [17]:
# Merge needs a key which must be identical in both data sets (here the key is 'bfs_number')
df3 = df2.merge(df_municip[['bfs_number', 
                            'pop', 
                            'pop_dens', 
                            'frg_pct', 
                            'emp']], 
                on="bfs_number")
df3.head(5)

,web-scraper-order,address_raw,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",3.0,49.0,0,1441.0,29.41,47.252171,8.845797,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0
1,1693998233-172,"Widacherstrasse 5, 8630 Rüti ZH, ZH",3.0,111.0,0,2600.0,23.42,47.252087,8.854919,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0
2,1693998256-331,"Widenweg 14, 8630 Rüti ZH, ZH",3.0,58.0,0,1490.0,25.69,47.253670,8.853993,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0
3,1693998265-381,"Rain 1, 8630 Rüti ZH, ZH",4.0,118.0,0,3240.0,27.46,47.259834,8.851705,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0
4,1693998276-419,"Bachtelstrasse 24b, 8630 Rüti ZH, ZH",3.0,66.0,0,1450.0,21.97,47.266113,8.866872,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0


### Join municipality data to rental apartment data using .merge() Task 2.j

In [23]:
# Merge needs a key which must be identical in both data sets (here the key is 'bfs_number')
df3 = df2.merge(df_municip[['bfs_number', 
                            'pop', 
                            'pop_dens', 
                            'frg_pct', 
                            'emp',
                            'mean_taxable_income']],
                on="bfs_number")
df3.head(5)

,web-scraper-order,address_raw,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,mean_taxable_income
0,1693998201-1,"Neuhusstrasse 6, 8630 Rüti ZH, ZH",3.0,49.0,0,1441.0,29.41,47.252171,8.845797,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0,65362.042683
1,1693998233-172,"Widacherstrasse 5, 8630 Rüti ZH, ZH",3.0,111.0,0,2600.0,23.42,47.252087,8.854919,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0,65362.042683
2,1693998256-331,"Widenweg 14, 8630 Rüti ZH, ZH",3.0,58.0,0,1490.0,25.69,47.253670,8.853993,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0,65362.042683
3,1693998265-381,"Rain 1, 8630 Rüti ZH, ZH",4.0,118.0,0,3240.0,27.46,47.259834,8.851705,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0,65362.042683
4,1693998276-419,"Bachtelstrasse 24b, 8630 Rüti ZH, ZH",3.0,66.0,0,1450.0,21.97,47.266113,8.866872,118,Rüti (ZH),12286,1221.272366,24.841283,5053.0,65362.042683


### Export data to file

In [24]:
# Remove missing values which may have ocured from the merging process
df3 = df3.dropna()

# Check for missing values per column
print(df3.isna().sum())

# Count number of rows and columns
print(df3.shape)

# Export apartment data to file
df3.to_csv('apartments_data_enriched.csv',
            sep=',',
            encoding='utf-8')

web-scraper-order      0
address_raw            0
rooms                  0
area                   0
luxurious              0
price                  0
price_per_m2           0
lat                    0
lon                    0
bfs_number             0
bfs_name               0
pop                    0
pop_dens               0
frg_pct                0
emp                    0
mean_taxable_income    0
dtype: int64
(816, 16)


### Sorting data

In [25]:
# Sorting data by 'price' and 'area' with highest price above (ascending=False)
df3.sort_values(by=['price', 'area'], 
                ascending=False).head()

,web-scraper-order,address_raw,rooms,area,luxurious,price,price_per_m2,lat,lon,bfs_number,bfs_name,pop,pop_dens,frg_pct,emp,mean_taxable_income
85,1693998265-365,"Gustav-Gull-Platz 4, 8004 Zürich, ZH",4.5,140.0,0,9950.0,71.07,47.379471,8.531120,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704
86,1693998265-366,"Engweg 8, 8006 Zürich, ZH",4.5,140.0,0,9170.0,65.50,47.385265,8.537632,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704
46,1693998236-207,"Etzelstrasse 35, 8038 Zürich, ZH",3.5,100.0,1,9150.0,91.50,47.346546,8.530695,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704
44,1693998236-197,"Utoquai 37, 8008 Zürich, ZH",3.5,169.0,0,9000.0,53.25,47.363682,8.546781,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704
117,1693998285-475,"Universitätstrasse 41, 8006 Zürich, ZH",4.5,140.0,0,8990.0,64.21,47.380440,8.547943,261,Zürich,420217,4778.994655,32.458468,491193.0,82932.737704


### Aggregation of data

In [26]:
# Aggregation using .groupby()
df3[['rooms', 'price']].groupby(['rooms']).mean()

,price
rooms,
1.0,1306.346154
1.5,2092.054054
2.0,2138.775000
2.5,2414.993056
3.0,1950.434783
3.5,2664.813084
4.0,2798.200000
4.5,3065.211765
5.0,2773.750000


## Reshaping data

### Reshaping data using .stack() and .unstack()

In [11]:
df_sub = df3[['bfs_name', 'rooms', 'price', 'area']][:5]
print('Original shape')
print(df_sub, '\n')

df_sub_stacked = df_sub.stack()
print('Stacked')
print(df_sub_stacked, '\n')

# Using unstack
print('Unstacked (= back to original shape)')
print(df_sub_stacked.unstack())

Original shape
    bfs_name  rooms   price   area
0  Rüti (ZH)    3.0  1441.0   49.0
1  Rüti (ZH)    3.0  2600.0  111.0
2  Rüti (ZH)    3.0  1490.0   58.0
3  Rüti (ZH)    4.0  3240.0  118.0
4  Rüti (ZH)    3.0  1450.0   66.0 

Stacked
0  bfs_name    Rüti (ZH)
   rooms             3.0
   price          1441.0
   area             49.0
1  bfs_name    Rüti (ZH)
   rooms             3.0
   price          2600.0
   area            111.0
2  bfs_name    Rüti (ZH)
   rooms             3.0
   price          1490.0
   area             58.0
3  bfs_name    Rüti (ZH)
   rooms             4.0
   price          3240.0
   area            118.0
4  bfs_name    Rüti (ZH)
   rooms             3.0
   price          1450.0
   area             66.0
dtype: object 

Unstacked (= back to original shape)
    bfs_name rooms   price   area
0  Rüti (ZH)   3.0  1441.0   49.0
1  Rüti (ZH)   3.0  2600.0  111.0
2  Rüti (ZH)   3.0  1490.0   58.0
3  Rüti (ZH)   4.0  3240.0  118.0
4  Rüti (ZH)   3.0  1450.0   66.0


### Reshaping data using .melt()

In [12]:
df_sub = df3[['rooms', 'price', 'area']][:5]
print('Original shape')
print(df_sub, '\n')

print('Reshaped using .melt()')
print(pd.melt(df, id_vars=['rooms'], value_vars=['price', 'area']))

Original shape
   rooms   price   area
0    3.0  1441.0   49.0
1    3.0  2600.0  111.0
2    3.0  1490.0   58.0
3    4.0  3240.0  118.0
4    3.0  1450.0   66.0 

Reshaped using .melt()
      rooms variable   value
0       3.0    price  1441.0
1       3.5    price  1850.0
2       2.0    price  4853.0
3       2.0    price  4335.0
4       2.0    price  3515.0
...     ...      ...     ...
1725    3.5     area    82.0
1726    4.0     area    73.0
1727    4.5     area   110.0
1728    3.0     area    68.0
1729    4.5     area   114.0

[1730 rows x 3 columns]


### Pivoting data using .pivot_table()

In [13]:
# Using pivot_table to reshape the data and calculate means 
pd.pivot_table(df3[['rooms', 'price', 'area']],
               index=['rooms'],
               values=['price', 'area', 'rooms'],
               aggfunc=(np.mean))

,area,price
rooms,,
1.0,32.057692,1306.346154
1.5,35.972973,2092.054054
2.0,54.475000,2138.775000
2.5,67.340278,2414.993056
3.0,67.956522,1950.434783
3.5,89.500000,2664.813084
4.0,96.900000,2798.200000
4.5,114.670588,3065.211765
5.0,103.500000,2773.750000


### Jupyter notebook --footer info-- (please always provide this at the end of each submitted notebook)

In [27]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('-----------------------------------')

-----------------------------------
POSIX
Darwin | 23.0.0
Datetime: 2023-10-06 17:09:57
Python Version: 3.9.6
-----------------------------------
